# Пример для **Chain of Responsibility pattern**. "Подтверждение скидки на товары электроники" 

1. **Проверка наличия товара (class ProductHandler):**
   - Запросить базу данных, чтобы проверить, есть ли еще товар на складе.
   - Если товара нет в наличии, отображается сообщение об ошибке.

2. **Подтвержение срока действия скидки (class DiscountHandler):**
   - Сравнение текущей даты с датами начала и окончания действия скидки.
   - Действуем дальше только в том случае, если текущая дата попадает в период действия скидки; в противном случае отобразите сообщение.

3. **Проверка величины скидки(class DiscountHandler):**
   - Извлечение цены продукта и величины скидки из базы данных.
   - Убедимся, что сумма скидки не превышает цену товара.
   - Если превышает, изменить величину скидки или вывести предупреждение.

4. **Проверка ограничений на определенные характеристики продукта(class BrandHandler):**
   - Извлечение бренда продукта из базы данных.
   - Убедиться, что этот бренд не включен в список запрещенных
   - Если включен в чпиков запрещенных, отобразить предупреждение.
  
5. **Уведомлять о малом количестве запасов товара(class NotificationHandler)**

  
В нашем примере база данных заменена словарем.


In [ ]:
import random
from datetime import datetime, timedelta
from abc import ABC, abstractmethod
from typing import Optional, Any
import logging

class Handler(ABC):
    @abstractmethod
    def set_next(self, handler: 'Handler') -> 'Handler':
        #Устанавливает следующий обработчик(handler) в цепочке.
        pass

    @abstractmethod
    def handle(self, request) -> Optional[str]:
        #Обрабатывает запрос.
        pass

class AbstractHandler(Handler):
    _next_handler: Handler = None

    def set_next(self, handler: Handler) -> Handler:
        #Устанавливает следующий обработчик(handler) в цепочке.
        self._next_handler = handler
        return handler

    def handle(self, product: Any) -> str:
        #Базовая реализация для обработки запросов.
        if self._next_handler:
            return self._next_handler.handle(product)
        return "You can use discount for this product"

class ProductHandler(AbstractHandler):
    def handle(self, product: dict) -> str:
        #Обрабатывает запросы на продукцию.
        if product['stock'] == 0:
            return f"There's no stock for {product['name']}"
        return super().handle(product)

class BrandHandler(AbstractHandler):
    _blocked_brands = ("apple",)

    def handle(self, product: dict) -> str:
        #Обрабатывает запросы, связанные с брендом.
        if product['brand'] in self._blocked_brands:
            return f"{product['name']} is not allowed to have a discount due to brand restrictions"
        return super().handle(product)

class Discount:
    @staticmethod
    def find(discount_id: int, discounts: dict) -> dict:
        #Метод-заполнитель для поиска скидки по ее идентификатору.
        return discounts.get(discount_id, None)

class DiscountHandler(AbstractHandler):
    def handle(self, product: dict) -> str:
        #Обрабатывает запросы, связанные со скидками.
        discount = Discount.find(product['id'],discount_dict)
        if not self.has_valid_period(discount['expires_at']):
            return "The discount has expired"

        if not self.has_valid_amount(product['price'], discount['amount']):
            return "Cannot apply discount; discount value exceeds product price"
        
        return super().handle(product)

    def has_valid_period(self, discount_expires_at):
        return discount_expires_at < datetime.now()

    def has_valid_amount(self, product_price, discount_value):
        return discount_value < product_price

class NotificationHandler(AbstractHandler):
    def handle(self, product: dict) -> str:
        #Обрабатывает уведомления о малом количестве товара на складе.
        if product['stock'] <= 10 and product['price'] >= 1000:
            logger = logging.getLogger(__name__)
            logger.info(f"{product['name']} is running out of stock; current stock: {product['stock']}")
        return super().handle(product)

# Генерирует случайные данные о продукте
def generate_random_product(product_id):
    id = product_id
    names = ["mobile phone", "laptop", "smartwatch", "tablet", "camera"]
    name = random.choice(names)  # Случайное название продукта
    stock = random.randint(0, 100)  # Случайное количество на складе
    brands = ["Samsung", "Apple", "Sony", "HP", "Dell"]
    brand = random.choice(brands)  # Случайная марка товара
    price = round(random.uniform(6000, 89000), 2)  # Случайная цена товара

    return {
        'id': id,
        'name': name,
        'stock': stock,
        'brand': brand,
        'price': price,
    }

# Генерирует случайные данные о скидках
def generate_random_discount(discount_id):
    id = discount_id
    amount = round(random.uniform(6000, 89000), 2)  # Случайная величина скидки
    expires_in_days = random.randint(-30, 30)  # Случайное количество дней до истечения срока действия скидки
    expires_at = datetime.now() + timedelta(days=expires_in_days)

    return {
        'id': id,
        'amount': amount,
        'expires_at': expires_at,
    }

# Создаем словарь из 5 записей
discount_dict = {}
for i in range(1,6):
    discount_id = i
    product_id = i
    discount_dict[discount_id] = generate_random_discount(discount_id)
    product_dict[product_id] = generate_random_product(product_id)

# Выводим сгенерированные данные о продукте
for product_id, p_data in product_dict.items():
    print(f"Product ID {product_id}: {p_data}")
print()
    
#Выводим сгенерированные данные о скидках
for discount_id, d_data in discount_dict.items():
    print(f"Discount ID {discount_id}: {d_data}")
print()


def main():
    for product_id, product_data in product_dict.items():
        main_product_handler = ProductHandler()
        main_brand_handler = BrandHandler()
        main_discount_handler = DiscountHandler()
        main_notification_handler = NotificationHandler()

        main_product_handler.set_next(main_brand_handler).set_next(main_discount_handler).set_next(main_notification_handler)

        result = main_product_handler.handle(product_data)
        print(f"Product ID {product_id}: {result}")

if __name__ == "__main__":
    main()

Product ID 1: {'id': 1, 'name': 'laptop', 'stock': 69, 'brand': 'Samsung', 'price': 11570.95}
Product ID 2: {'id': 2, 'name': 'smartwatch', 'stock': 0, 'brand': 'Apple', 'price': 15937.38}
Product ID 3: {'id': 3, 'name': 'tablet', 'stock': 71, 'brand': 'Dell', 'price': 7893.24}
Product ID 4: {'id': 4, 'name': 'laptop', 'stock': 0, 'brand': 'Samsung', 'price': 55182.73}
Product ID 5: {'id': 5, 'name': 'camera', 'stock': 22, 'brand': 'HP', 'price': 46147.73}

Discount ID 1: {'id': 1, 'amount': 59643.38, 'expires_at': datetime.datetime(2024, 5, 4, 12, 21, 36, 781000)}
Discount ID 2: {'id': 2, 'amount': 80183.93, 'expires_at': datetime.datetime(2024, 4, 7, 12, 21, 36, 781000)}
Discount ID 3: {'id': 3, 'amount': 71087.5, 'expires_at': datetime.datetime(2024, 5, 1, 12, 21, 36, 781000)}
Discount ID 4: {'id': 4, 'amount': 88727.48, 'expires_at': datetime.datetime(2024, 5, 18, 12, 21, 36, 781000)}
Discount ID 5: {'id': 5, 'amount': 65003.84, 'expires_at': datetime.datetime(2024, 5, 18, 12, 21, 